# Fantasy Bundesliga
## Initial Idea

I enjoy playing [Fantasy Bundesliga](https://fantasy.bundesliga.com/). However, I don't have the time, nor the motivation to keep myself up to date on whatever is happening in today's soccer world. 

I had found a blog post by [Philip Kalinda](http://www.philipkalinda.com/ds9.html) as inspiration. The basic idea is to use linear optimization to find the line up that maximizes the team's points. Unfortunately, there is no official API to fetch the official Bundesliga data. Even more unfortunately, there doesn't exist an official historical database for Fantasy Bundesliga.

### Basic Rules of Fantasy Bundesliga

You'll need to select a soccer team of 15 players, consisting of:

* 2 Goalkeepers
* 5 Defenders
* 5 Midfielders
* 3 Forwards

You're allowed to select players from any Bundesliga club. However, you have a budget of 150 million. Based upon each player's real Bundesliga match stats, you'll earn points.

### Fantasy Data

Unfortunately, there is no official API that you can use to fetch the Fantasy data. Additionally, there is no official historical database that can be used as a reference to set up your first squad. The only resource I was able to find is by [Statbunker](https://www.statbunker.com/competitions/FantasyFootballPlayersStats?comp_id=646). With the help of Selenium, I'll scrape last season's Bundesliga performance for each player, and each player's price from the official Fantasy Bundesliga website. Both datasets will be merged and will make up the basis of the linear optimization.

## Scraping the data

In [2]:
import os
import re
import time

import pandas as pd
import pulp
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

In [10]:
class HistoricalData:
    """
    Scrapes Statbunker to get all historic performance of players in Bundesliga season 2019/20.

    Args:
        driver (webdriver.Chrome): Webdriver object.
        data (pd.DataFrame): Scraped historic data.
    """

    def __init__(self):
        self.driver = self.set_driver()
        self.data = self.fetch_data()
        self.driver.quit()

    def set_driver(self) -> webdriver.Chrome:
        """
        Creates the webdriver and sets its options.

        Returns:
            driver (webdriver.Chrome): webdriver object.
        """

        options = webdriver.ChromeOptions()

        dir_current = os.getcwd()
        prefs = {'download.default_directory': dir_current}
        options.add_experimental_option('prefs', prefs)

        options.add_argument('--headless')
        options.add_argument('--window-size=1920x1080')

        driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

        return driver

    def fetch_data(self) -> pd.DataFrame:
        """
        Fetches the data from Statbunker and creates a DataFrame object.

        Returns:
            data (pd.DataFrame): DataFrame containing the Statbunker data.
        """

        url = 'https://www.statbunker.com/competitions/FantasyFootballPlayersStats?comp_id=646'

        cols = ['Name', 'Points', 'Clubs', 'Position']
        data = pd.DataFrame(columns=cols)

        self.driver.get(url)

        fantasy_table = self.driver.find_element_by_xpath('//table[@class="table"]/tbody')

        for row in fantasy_table.find_elements_by_xpath('.//tr'):
            data_row = pd.Series([td.text for td in row.find_elements_by_xpath(".//td")][:4], index=cols)
            data = data.append(data_row, ignore_index=True)

        data = data.drop_duplicates(keep='first')
        data = data.reset_index(drop=True)

        return data

In [11]:
statbunker = HistoricalData()

[WDM] - Current google-chrome version is 81.0.4044
[WDM] - Get LATEST driver version for 81.0.4044
[WDM] - Driver [/Users/a733129/.wdm/drivers/chromedriver/mac64/81.0.4044.138/chromedriver] found in cache


In [12]:
statbunker.data

,Name,Points,Clubs,Position
0,Robert Lewandowski,239,BM1,Forward
1,Timo Werner,227,,Forward
2,Jadon Sancho,188,BD1,Midfielder
3,Thomas Muller,157,BM1,Forward
4,Serge Gnabry,147,BM1,Midfielder
...,...,...,...,...
567,Samuel Fridjonsson,-2,,Defender
568,Jan-Luca Rumpf,-2,,Defender
569,Matthias Bader,-2,FCK1,Defender
570,Malick Thiaw,-3,SCH4,Defender


In [66]:
class BundesligaData:
    """
    Scrapes the Bundesliga Fantasy page to get the initial transfer data for each available player.

    Args:
        cols (list): Columns for the scraped data
        gdpr (bool): Indicator if GDPR popup has been accepted.
    """

    def __init__(self):

        self.cols = ['Name', 'Position', 'Price']
        self.gdpr = False

    def fetch_data_pipeline(self, username: str, password: str):
        """
        Pipeline that fetches the Bundesliga Fantasy data:
            1. Creates webdriver object.
            2. Logs into the website with the given credentials.
            3. Fetches data from the website.
            4. Quits webdriver object.

        Args:
            username (str): Username that can be used to login.
            password (str): Password that can be used to login.
        """

        print('Fetching data started.')

        self.driver = self.set_driver()

        self.login(username, password)

        self.bundesliga_data = self.pull_transfer_data()

        self.driver.quit()

        print('Fetching data finished.')

    def set_driver(self) -> webdriver.Chrome:
        """
        Creates the webdriver and sets its options.

        Returns:
            driver (webdriver.Chrome): webdriver object.
        """

        options = webdriver.ChromeOptions()

        dir_current = os.getcwd()
        prefs = {'download.default_directory': dir_current}
        options.add_experimental_option('prefs', prefs)

        options.add_argument('--headless')
        options.add_argument('--window-size=1920x1080')

        driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

        return driver

    def login(self, username: str, password: str):
        """
        Logs into the Bundesliga website.

        Args:
            username (str): Username that can be used to login.
            password (str): Password that can be used to login.
        """

        print('Login started.')

        url = 'https://fantasy.bundesliga.de/?register_or_login=login'

        self.driver.get(url)

        WebDriverWait(self.driver, 15).until(EC.presence_of_element_located((By.NAME, 'username')))

        self.driver.find_elements_by_name('username')[2].clear()
        self.driver.find_elements_by_name('username')[2].send_keys(username)

        self.driver.find_elements_by_name('password')[3].clear()
        self.driver.find_elements_by_name('password')[3].send_keys(password)

        self.driver.find_element_by_xpath('//input[@value="Anmelden"]').click()

        WebDriverWait(self.driver, 15).until(EC.url_changes(url))

        print('Login finished.')

    def check_gdpr(self):
        """
        Checks if the GDPR popup has been accepted.
        """

        if not self.gdpr:
            WebDriverWait(self.driver, 15).until(EC.presence_of_element_located((By.ID, 'onetrust-accept-btn-handler')))
            self.driver.find_element_by_id('onetrust-accept-btn-handler').click()
            WebDriverWait(self.driver, 15).until(
                EC.invisibility_of_element_located((By.ID, 'onetrust-accept-btn-handler')))
            self.gdpr = True

    def pull_transfer_data(self) -> pd.DataFrame:
        """
        Navigates the page that will be scraped. Iterates through the pages of the transfer information table.

        Returns:
            data (pd.DataFrame): DataFrame containing all the transfer information data.
        """

        print('Scraping table started.')

        url = 'https://fantasy.bundesliga.de/player_transfers'

        self.driver.get(url)
        time.sleep(1)

        try:
            self.check_gdpr()

        except TimeoutException:
            pass

        data = self.pull_transfer_data_table()

        for offset in range(25, 525, 25):

            time.sleep(1)
            xpath_offset = '//div[@data-offset=' + str(offset) + ']'

            while True:

                next_button = fantasy.driver.find_element_by_xpath(xpath_offset)
                next_button.click()
                time.sleep(1)

                table = self.pull_transfer_data_table()

                if fantasy.driver.find_element_by_xpath(
                        '//div[@class="paginator__item js-update-offset active"]').get_attribute('data-offset') == str(
                    offset):
                    break

            data = pd.concat([data, table], axis=0)

        data = data.drop_duplicates(keep='first')
        data = data.reset_index(drop=True)

        print('Scraping table finished.')

        return data

    def pull_transfer_data_table(self) -> pd.DataFrame:
        """
        Fetches the data from the transfer information table that is currently visible on the page.

        Returns:
             fantasy_data (pd.DataFrame): Scraped information of the currently present table.
        """

        fantasy_table = self.driver.find_element_by_xpath('//div[@class="players-list"]')
        fantasy_data = pd.DataFrame(columns=self.cols)

        names = fantasy_table.find_elements_by_xpath('.//div[@class="transf__playerInList__nameSection"]/a')
        positions = fantasy_table.find_elements_by_xpath(
            './/div[@class="transf__playerInList__nameSection"]/span[@class="player-info__position"]')
        prices = fantasy_table.find_elements_by_xpath('.//div[@class="transf__playerInList__prize"]')

        for name, position, price in zip(names, positions, prices):
            row = pd.Series([name.text, position.text,
                             price.text.split(' ')[0]],
                            index=self.cols)

            fantasy_data = fantasy_data.append(row, ignore_index=True)

        return fantasy_data

In [67]:
username = 'XYZ'
password = 'XYZ'
fantasy = BundesligaData()
fantasy.fetch_data_pipeline(username, password)

[WDM] - Current google-chrome version is 81.0.4044
[WDM] - Get LATEST driver version for 81.0.4044
[WDM] - Driver [/Users/a733129/.wdm/drivers/chromedriver/mac64/81.0.4044.138/chromedriver] found in cache


Fetching data started.
 
Login started.
Login finished.
Scraping table started.
Scraping table finished.
Fetching data finished.


In [3]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [193]:
fantasy.data['Name'] = fantasy.data.Name.apply(lambda row: remove_accents(row))
data_merged = fantasy.data.merge(statbunker.data[['Name', 'Points']], how='inner', on='Name')
data_merged.Points = data_merged.Points.astype(int)
data_merged.Price = data_merged.Price.astype(int)
data_merged = data_merged.drop_duplicates(subset='Name', keep='first')
data_merged = data_merged.reset_index(drop=True)
data_merged.to_csv('data/fantasy_data.csv', index=False)

## Linear Optimization

Let's summarize the problem, I am trying to solve: 

* I want to maximize the points my team will generate

But I have a few constraints when selecting my players, such as:

* I can only spend a maximum of 150 million
* I need to choose 2 goalkeepers
* I need to choose 5 defenders
* I need to choose 5 midfielders
* I need to choose 3 forwards

So we have a formula that we want to maximize (points), and we have a few constraints (e.g. a budget of 150 million). An almost perfect set up for some linear optimization.

We create our decision variables:

In [ ]:
def create_dec_var(data: pd.DataFrame) -> list:
    """
    Creates the decision variables for the optimization problem.

    Args:
         data (pd.DataFrame): DataFrame containing the Fantasy data.

    Returns:
        decision_variables (list): List of all decision variables.
    """

    decision_variables = []

    for rownum, row in data.iterrows():
        variable = str('x' + str(rownum))
        variable = pulp.LpVariable(str(variable), lowBound=0, upBound=1, cat='Integer')
        decision_variables.append(variable)

    return decision_variables

This is our objective function. The function we want to maximize.

In [ ]:
def total_points(data: pd.DataFrame, lst: list, problem: pulp.pulp.LpProblem) -> pulp.pulp.LpProblem:
    """
    Creates the objective function for the optimization problem.

    Args:
         data (pd.DataFrame): DataFrame containing the Fantasy data.
         lst (list): List of all decision variables.
         problem (pulp.pulp.LpProblem): Problem set of the optimization problem.

    Returns:
        problem (pulp.pulp.LpProblem): Adjusted problem set of the optimization problem.
    """
    
    total_points = ""
    for rownum, row in data.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                formula = row['Points'] * player
                total_points += formula
    
    problem += total_points
    
    return problem

And our constraints:

In [145]:
def cash(data: pd.DataFrame, lst: list, problem: pulp.pulp.LpProblem, cash: int) -> pulp.pulp.LpProblem:
    """
    Creates the budget constraint for the optimization problem.

    Args:
         data (pd.DataFrame): DataFrame containing the Fantasy data.
         lst (list): List of all decision variables.
         problem (pulp.pulp.LpProblem): Problem set of the optimization problem.

    Returns:
        problem (pulp.pulp.LpProblem): Adjusted problem set of the optimization problem.
    """

    total_paid = ""
    for rownum, row in data.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                formula = row['Price'] * player
                total_paid += formula
    problem += (total_paid <= cash), "Cash"

    return problem


def team_gkp(data: pd.DataFrame, lst: list, problem: pulp.pulp.LpProblem, avail_gk: int) -> pulp.pulp.LpProblem:
    """
    Creates the number of goalkeepers constraint for the optimization problem.

    Args:
         data (pd.DataFrame): DataFrame containing the Fantasy data.
         lst (list): List of all decision variables.
         problem (pulp.pulp.LpProblem): Problem set of the optimization problem.

    Returns:
        problem (pulp.pulp.LpProblem): Adjusted problem set of the optimization problem.
    """

    total_gk = ""
    for rownum, row in data.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                if row['Position'] == 'TW':
                    formula = 1 * player
                    total_gk += formula

    problem += (total_gk == avail_gk), "TW"

    return problem


def team_def(data: pd.DataFrame, lst: list, problem: pulp.pulp.LpProblem, avail_def: int) -> pulp.pulp.LpProblem:
    """
    Creates the number of defenders constraint for the optimization problem.

    Args:
         data (pd.DataFrame): DataFrame containing the Fantasy data.
         lst (list): List of all decision variables.
         problem (pulp.pulp.LpProblem): Problem set of the optimization problem.

    Returns:
        problem (pulp.pulp.LpProblem): Adjusted problem set of the optimization problem.
    """

    total_def = ""
    for rownum, row in data.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                if row['Position'] == 'VT':
                    formula = 1 * player
                    total_def += formula

    problem += (total_def == avail_def), "VT"

    return problem


def team_mid(data: pd.DataFrame, lst: list, problem: pulp.pulp.LpProblem, avail_mid: int) -> pulp.pulp.LpProblem:
    """
    Creates the number of midfielders constraint for the optimization problem.

    Args:
         data (pd.DataFrame): DataFrame containing the Fantasy data.
         lst (list): List of all decision variables.
         problem (pulp.pulp.LpProblem): Problem set of the optimization problem.

    Returns:
        problem (pulp.pulp.LpProblem): Adjusted problem set of the optimization problem.
    """

    total_mid = ""
    for rownum, row in data.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                if row['Position'] == 'MI':
                    formula = 1 * player
                    total_mid += formula

    problem += (total_mid == avail_mid), "MI"

    return problem


def team_fwd(data: pd.DataFrame, lst: list, problem: pulp.pulp.LpProblem, avail_fwd: int) -> pulp.pulp.LpProblem:
    """
    Creates the number of forwards constraint for the optimization problem.

    Args:
         data (pd.DataFrame): DataFrame containing the Fantasy data.
         lst (list): List of all decision variables.
         problem (pulp.pulp.LpProblem): Problem set of the optimization problem.

    Returns:
        problem (pulp.pulp.LpProblem): Adjusted problem set of the optimization problem.
    """

    total_fwd = ""
    for rownum, row in data.iterrows():
        for i, player in enumerate(lst):
            if rownum == i:
                if row['Position'] == 'ST':
                    formula = 1 * player
                    total_fwd += formula

    problem += (total_fwd == avail_fwd), "ST"

    return problem

Then we assemble, and define it as a maximization problem.

In [146]:
def find_prob(data: pd.DataFrame, cash: int, gk: int, de: int, mi: int, fw: int) -> pulp.pulp.LpProblem:
    """
    Assembles the whole problem set for the optimization problem.

    Args:
         data (pd.DataFrame): DataFrame containing the Fantasy data.
         cash (int): Available budget.
         gk (int): Number of goalkeeprs.
         de (int): Number of defenders.
         mi (int): Number of midfielders.
         fw (int): Number of forwards.

    Returns:
        problem (pulp.pulp.LpProblem): Adjusted problem set of the optimization problem.
    """

    problem = pulp.LpProblem('FantasyTeam', pulp.LpMaximize)
    lst = create_dec_var(data)

    problem = total_points(data, lst, problem)
    problem = cash(data, lst, problem, cash)
    problem = team_gkp(data, lst, problem, gk)
    problem = team_def(data, lst, problem, de)
    problem = team_mid(data, lst, problem, mi)
    problem = team_fwd(data, lst, problem, fw)

    return problem


def LP_optimize(data: pd.DataFrame, problem: pulp.pulp.LpProblem):
    """
    Sets up optimization problem.

    Args:
        data (pd.DataFrame): DataFrame containing the Fantasy data.
        problem (pulp.pulp.LpProblem): Problem set of the optimization problem.
    """

    problem.writeLP('FantasyTeam.lp')

    optimization_result = problem.solve()

    print("Status:", pulp.LpStatus[problem.status])
    print("Optimal value:", pulp.value(problem.objective))

    assert optimization_result == pulp.LpStatusOptimal

Our decision function will find the optimum. It will return our orignial DataFrame with an additional column called *Decision* that will highlight the (non-) selected players.

In [158]:
def df_decision(data: pd.DataFrame, problem: pulp.pulp.LpProblem) -> pd.DataFrame:
    """
    Converts optimization results into readable DataFrame.

    Args:
        data (pd.DataFrame): DataFrame containing the Fantasy data.
        problem (pulp.pulp.LpProblem): Problem set of the optimization problem.

    Returns:
        data (pd.DataFrame): Original DataFrame that contains the Fantasy data plus column 'Decision' that
                indicates which players optimize the total points.
    """

    variable_name = []
    variable_value = []

    for v in problem.variables():
        variable_name.append(v.name)
        variable_value.append(v.varValue)

    df_vals = pd.DataFrame({'variable': variable_name, 'value': variable_value})

    for rownum, row in df_vals.iterrows():
        value = re.findall(r'(\d+)', row['variable'])
        df_vals.loc[rownum, 'variable'] = int(value[0])

    for rownum, row in data.iterrows():
        for results_rownum, results_row in df_vals.iterrows():
            if rownum == results_row['variable']:
                data.loc[rownum, 'Decision'] = results_row['value']

    return data

Only the points of 11 selected players in your team will count towards your score. To find the best possible line-up, we will *fill* our bench with players that cost the minium (1 million). So for example, if we are trying to find the line up that maximizes points for the 3-5-2 formation, we will assum that the 2 defenders on the bench will cost only 1 million each. Alas, the budget is 146 million as there are always 4 substitute players.

In [186]:
def find_best_formation(data: pd.DataFrame, cash: int, gk: int, de: int, mi: int, fw: int) -> pd.DataFrame:
    """
    Finds the best formation for a given formation, plus the solutions total points.

    Args:
         data (pd.DataFrame): DataFrame containing the Fantasy data.
         cash (int): Available budget.
         gk (int): Number of goalkeeprs.
         de (int): Number of defenders.
         mi (int): Number of midfielders.
         fw (int): Number of forwards.

    Returns:
        data_filtered (pd.DataFrame): DataFrame that shows the optimal line up for a given formation.
    """

    problem = find_prob(data, cash, gk, de, mi, fw)
    LP_optimize(data, problem)
    data_final = df_decision(data, problem)

    print('Total Points: {}'.format(data_final[data_final.Decision == 1].Points.sum()))

    sort_dict = {'TW': 0, 'VT': 1, 'MI': 2, 'ST': 3}

    data_final['Position'] = pd.Categorical(data_final['Position'],
                                            categories=sorted(sort_dict, key=sort_dict.get),
                                            ordered=True)

    data_filtered = data_final[data_final.Decision == 1].sort_values(by='Position')

    return data_filtered

In [187]:
find_best_formation(data_merged, 150-4, 1, 3, 4, 3)

Total Points: 1537


,Name,Position,Price,Points,Decision
24,Manuel Neuer,TW,13.0,118,1.0
7,Benjamin Pavard,VT,15.0,132,1.0
22,Raphael Guerreiro,VT,14.0,121,1.0
62,Christian Gunter,VT,11.0,108,1.0
3,Jadon Sancho,MI,17.0,188,1.0
18,Thorgan Hazard,MI,14.0,132,1.0
21,Marcel Sabitzer,MI,14.0,124,1.0
91,Davy Klaassen,MI,10.0,102,1.0
0,Robert Lewandowski,ST,18.0,239,1.0
85,Florian Niederlechner,ST,10.0,141,1.0


In [188]:
find_best_formation(data_merged, 150-4, 1, 3, 5, 2)

Total Points: 1503


,Name,Position,Price,Points,Decision
24,Manuel Neuer,TW,13.0,118,1.0
7,Benjamin Pavard,VT,15.0,132,1.0
22,Raphael Guerreiro,VT,14.0,121,1.0
33,Lukas Klostermann,VT,12.0,109,1.0
3,Jadon Sancho,MI,17.0,188,1.0
18,Thorgan Hazard,MI,14.0,132,1.0
21,Marcel Sabitzer,MI,14.0,124,1.0
91,Davy Klaassen,MI,10.0,102,1.0
132,Marius Bulter,MI,8.0,97,1.0
0,Robert Lewandowski,ST,18.0,239,1.0


In [189]:
find_best_formation(data_merged, 150-4, 1, 4, 3, 3)

Total Points: 1529


,Name,Position,Price,Points,Decision
24,Manuel Neuer,TW,13.0,118,1.0
7,Benjamin Pavard,VT,15.0,132,1.0
22,Raphael Guerreiro,VT,14.0,121,1.0
33,Lukas Klostermann,VT,12.0,109,1.0
62,Christian Gunter,VT,11.0,108,1.0
3,Jadon Sancho,MI,17.0,188,1.0
18,Thorgan Hazard,MI,14.0,132,1.0
132,Marius Bulter,MI,8.0,97,1.0
0,Robert Lewandowski,ST,18.0,239,1.0
15,Wout Weghorst,ST,14.0,144,1.0


In [190]:
find_best_formation(data_merged, 150-4, 1, 4, 4, 2)

Total Points: 1509


,Name,Position,Price,Points,Decision
24,Manuel Neuer,TW,13.0,118,1.0
7,Benjamin Pavard,VT,15.0,132,1.0
22,Raphael Guerreiro,VT,14.0,121,1.0
33,Lukas Klostermann,VT,12.0,109,1.0
62,Christian Gunter,VT,11.0,108,1.0
3,Jadon Sancho,MI,17.0,188,1.0
18,Thorgan Hazard,MI,14.0,132,1.0
21,Marcel Sabitzer,MI,14.0,124,1.0
132,Marius Bulter,MI,8.0,97,1.0
0,Robert Lewandowski,ST,18.0,239,1.0


In [191]:
find_best_formation(data_merged, 150-4, 1, 4, 5, 1)

Total Points: 1470


,Name,Position,Price,Points,Decision
24,Manuel Neuer,TW,13.0,118,1.0
7,Benjamin Pavard,VT,15.0,132,1.0
22,Raphael Guerreiro,VT,14.0,121,1.0
33,Lukas Klostermann,VT,12.0,109,1.0
62,Christian Gunter,VT,11.0,108,1.0
3,Jadon Sancho,MI,17.0,188,1.0
18,Thorgan Hazard,MI,14.0,132,1.0
21,Marcel Sabitzer,MI,14.0,124,1.0
91,Davy Klaassen,MI,10.0,102,1.0
132,Marius Bulter,MI,8.0,97,1.0


In [192]:
find_best_formation(data_merged, 150-4, 1, 5, 4, 1)

Total Points: 1455


,Name,Position,Price,Points,Decision
24,Manuel Neuer,TW,13.0,118,1.0
7,Benjamin Pavard,VT,15.0,132,1.0
22,Raphael Guerreiro,VT,14.0,121,1.0
33,Lukas Klostermann,VT,12.0,109,1.0
62,Christian Gunter,VT,11.0,108,1.0
133,Manuel Akanji,VT,8.0,82,1.0
3,Jadon Sancho,MI,17.0,188,1.0
18,Thorgan Hazard,MI,14.0,132,1.0
21,Marcel Sabitzer,MI,14.0,124,1.0
91,Davy Klaassen,MI,10.0,102,1.0


From all the possible solutions, we can see that a 3-4-3 line up maximizes the objective function with a total of 1537 points. 

The optimal line up looks like this:
    
**Goalkeeper:**
* Manuel Neuer

**Defender:**
* Benjamin Pavard
* Raphael Guerreiro
* Christian Gunter

**Midfielder:**
* Jadon Sancho
* Thorgan Hazard
* Marcel Sabitzer
* Davy Klaassen

**Forwards:**
* Robert Lewandowski
* Florian Niederlechner
* Robin Quaison